##### Monte Carlo simulations for 2sls and two-step efficient GMM

###### Generate n = 500 observations for $(Yi,Xi,Zi)$ from the following IV regressionmodel with heteroskedastic errors:

In [140]:
# The libraries we need for this task
using Distributions, PrettyTables, Random, Parameters, DataFrames
using LinearAlgebra, Statistics

In [141]:

# This function generate data based on the model
function  generate_sample(n,β,γ,Π,ρ)

    # errors terms
    Σ = [1.0 ρ;ρ 1.0]
    mvnormal = MvNormal([0.0; 0.0], Σ)
    error = rand(mvnormal,n)'
    ϵ = error[:,1]
    V = error[:,2]

    # model variables
    Z = randn(n,2)
    U = exp.(Z*γ) .* ϵ
    X = Z*Π + V
    Y = β*X + U

    return (Y = Y, X = X, Z = Z)

end



generate_sample (generic function with 1 method)

In [142]:
# Specify the parameters of the model
β = 1.0
γ = Π = [1.0, 1.0]
ρ = 0.95

@unpack Y, X, Z = generate_sample(5000,β,γ,Π,ρ);

###### Computation of $β^{2SLS}$, the 2SLS estimator of $β$, and its standard error.

In [143]:
# This function compute an estimate of Ω.
function Ω(U,Z)
    n = length(Z)
    ZU = Z.*U
    return (ZU'*ZU)/n
end

Ω (generic function with 1 method)

In [144]:
# This function compute an estimate of β2sls and its standart error
function β₂ₛₗₛ(Y, X, Z)
    n = length(Y)
    # estimator 
    PZ = Z*(inv(Z'Z)*Z')
    β2sls = inv(X'*PZ*X)*(X'*PZ*Y)

    # standard error
    Q = (X'Z)/n
    A = inv((Z'Z)/n)
    Ωₙ = Ω(Y-X*β2sls,Z)
    Variance = (inv(Q*A*Q')*(Q*A*Ωₙ*A*Q'))*inv(Q*A*Q')
    std_err_2sls = sqrt(Variance/n)
    return(β2sls = β2sls, std_err_2sls = std_err_2sls)
end


β₂ₛₗₛ (generic function with 1 method)

In [145]:
@unpack β2sls, std_err_2sls = β₂ₛₗₛ(Y, X, Z)

(β2sls = 0.9301127525532763, std_err_2sls = 0.1622953844746321)

###### Computation of $β^{GMM}$, the GMM estimator of $β$, and its standard error.

In [146]:
# This function compute an estimate of βGMM and its standart error
function βᵍᵐᵐ(Y, X, Z)
    n = length(Y)

    # estimator 
    Q = (X'Z)/n
    P = (Z'Y)/n
    Ωₙ = Ω(Y-X*β2sls,Z)
    βgmm = inv((Q*inv(Ωₙ))*Q')*((Q*inv(Ωₙ))*P)

    # standard error
    Variance = inv((Q*inv(Ωₙ))*Q')
    std_gmm = sqrt(Variance/n)
    return(βgmm = βgmm, std_gmm = std_gmm)
end


βᵍᵐᵐ (generic function with 1 method)

In [147]:
@unpack βgmm, std_gmm = βᵍᵐᵐ(Y, X, Z)

(βgmm = 0.9407598111416822, std_gmm = 0.16201463289659399)

In [148]:
A = [1 2; 0 3; 1 -1]
u = [2,4,-1]
@show A
@show u
@show A.*u

A = [1 2; 0 3; 1 -1]
u = [2, 4, -1]
A .* u = [2 4; 0 12; -1 1]


3×2 Matrix{Int64}:
  2   4
  0  12
 -1   1